In [1]:
import pandas as pd; DF=pd.DataFrame
import requests
from tqdm.notebook import tqdm

### School Record Scrapper

In [2]:
### Scrap http://www.cfbstats.com/ for team records
### And save as CSV 
team_match_dict={
    'BYU':'Brigham Young',"Hawai'i":'Hawaii', 
    'Miami (Florida)':'Miami (Fla.)', 'SMU':'Southern Methodist', 
    'TCU':'Texas Christian', 'UAB':'Alabama at Birmingham', 
    'UCF':'Central Florida', 'UNLV':'Nevada-Las Vegas',
    'USC':'Southern California','UTEP':'Texas-El Paso', 
    'UTSA':'Texas-San Antonio',
}
def get_team_dfs_from_cfbstats(team_id):
    url = f'http://www.cfbstats.com/2019/team/{team_id}/index.html'
    r=requests.get(url)
    try:
        dfs=pd.read_html(r.content)
    except ValueError:
        dfs=None
    return dfs

def get_team_record_from_dfs(dfs):
    df1=dfs[0]
    df2=dfs[1]
    df3=dfs[2]
    teamName=df1.columns[1]
    ser=df3['Record'].str.split('-',expand=True).rename(columns={0:'W',1:'L'}).iloc[0]
    ser.name=teamName
    return ser
    
def get_record_df():
    record_df=DF()
    for i in tqdm(range(0,1000)):
        tid = str(i)
        dfs=get_team_dfs_from_cfbstats(tid)
        if not dfs: continue
        record = get_team_record_from_dfs(dfs)
        record_df=record_df.append(record)
    return record_df
def convert_name(name):
    team_match_dict={
        'BYU':'Brigham Young',"Hawai'i":'Hawaii', 
        'Miami (Florida)':'Miami (Fla.)', 'SMU':'Southern Methodist', 
        'TCU':'Texas Christian', 'UAB':'Alabama at Birmingham', 
        'UCF':'Central Florida', 'UNLV':'Nevada-Las Vegas',
        'USC':'Southern California','UTEP':'Texas-El Paso', 
        'UTSA':'Texas-San Antonio',
    }
    new_name= team_match_dict.get(name)
    if pd.isna(new_name):
        return name
    else:
        return new_name
def main():
    record_df = get_record_df()
    record_df = record_df.reset_index().rename(columns={'index':'School'})
    record_df['School']=record_df['School'].map(convert_name)
    record_df.to_csv('_records.csv',index=False)
main()